### Import packages 

In [195]:
import pandas as pd
from tabulate import tabulate
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier, RandomForestClassifier ,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , f1_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial import distance
from pathlib import Path
from skimage import data, util, measure
from skimage import data, util
from sklearn.preprocessing import StandardScaler
from skimage.measure import label, regionprops
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

### Import Data

In [196]:
df = pd.read_csv(r"D:\Self Study\Projects\titanic\train.csv")

In [197]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [198]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [200]:
def check_duplicate_columns(df):
    duplicated_columns = []
    names_set = set() 

    for i in range(len(df.columns)):
        for j in range(i + 1, len(df.columns)):
            if (df.iloc[:, i] == df.iloc[:, j]).all():
                # duplicated_columns[f"{i}"] = df.columns[i], df.columns[j]
                duplicated_columns.append((df.columns[i], df.columns[j]))
                names_set.add(df.columns[i])
                names_set.add(df.columns[j])

    return duplicated_columns, names_set

duplicated_columns,names_set = check_duplicate_columns(df)
print(duplicated_columns , names_set )
print('percents of duplicated columns : ', len(names_set)/len(df.columns))

# or simply use this
# df.columns.duplicates()

[] set()
percents of duplicated columns :  0.0


In [201]:
def unique_null_values(df):
    data = []
    for i in df.columns:
        unique = df.nunique()[i]
        data.append({
            'column_name': i,
            'length': len(df[i]),
            'num_unique': unique,
            'per_num_un': round(unique / len(df[i]), 4),
            "Valid_values":  len(df[i]) - df.isnull().sum()[i],
            "NULL_num": df.isnull().sum()[i],
            "per_Null_num": (df.isnull().sum()[i]/(len(df)))*100
        })

    return pd.DataFrame(data)

summary = unique_null_values(df)
summary

,column_name,length,num_unique,per_num_un,Valid_values,NULL_num,per_Null_num
0,PassengerId,891,891,1.0000,891,0,0.000000
1,Survived,891,2,0.0022,891,0,0.000000
2,Pclass,891,3,0.0034,891,0,0.000000
3,Name,891,891,1.0000,891,0,0.000000
4,Sex,891,2,0.0022,891,0,0.000000
5,Age,891,88,0.0988,714,177,19.865320
6,SibSp,891,7,0.0079,891,0,0.000000
7,Parch,891,7,0.0079,891,0,0.000000
8,Ticket,891,681,0.7643,891,0,0.000000
9,Fare,891,248,0.2783,891,0,0.000000


In [202]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


i want to try:
1-compare accuracy when i remove the 177 null values Rows vs removing the whole Age Features ???

In [203]:
cat_cols=df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols,"\n")
print("Numerical Variables:")
print(num_cols)

Categorical Variables:
Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object') 

Numerical Variables:
['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [204]:
df.dropna(subset=['Embarked'], inplace=True)

In [205]:
df['Sex'].value_counts()

male      577
female    312
Name: Sex, dtype: int64

In [206]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [207]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1,"Q": 2})

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,0
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,1


In [208]:
data = df.drop(['Name',"Cabin","Age","Survived","PassengerId","Ticket"], axis = 1)

In [209]:
print(df.groupby('Survived').size())

Survived
0    549
1    340
dtype: int64


In [210]:
X =  data
y = df.Survived

In [211]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [212]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [213]:
len(X_train) 

711

In [214]:
len(X_validation)

178

In [215]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.783392 (0.026201)
LDA: 0.774941 (0.029021)
KNN: 0.741119 (0.050024)
CART: 0.783314 (0.059048)
NB: 0.773592 (0.032731)
SVM: 0.748200 (0.050741)


try next option -> to remove age from training

In [216]:
data_2 = df.drop(['Name',"Cabin","PassengerId","Ticket"], axis = 1)

In [217]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    int64  
 3   Age       712 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Embarked  889 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 62.5 KB


In [218]:
data_2["Age"].isnull().sum()

177

In [219]:
data_2 = data_2.dropna()

In [220]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    int64  
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 50.1 KB


In [221]:
y = data_2.Survived
X  = data_2.drop(["Survived"], axis = 1)

In [222]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [223]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.794267 (0.043660)
LDA: 0.797744 (0.047448)
KNN: 0.673183 (0.065405)
CART: 0.767951 (0.047280)
NB: 0.783741 (0.039076)
SVM: 0.683615 (0.054579)


Removing one column (with null values) from 7 columns in this case made better accuracy than removing 177 from 889 rows 

what if i filled the gaps with Imputation techniques

In [224]:
# data_3= df['Age'].fillna(df['Age'].mean(), inplace=True)
# data_3 = df.drop(['Name',"Cabin","PassengerId","Ticket"], axis = 1)
# data_3.info()


In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    int64  
 5   Age          712 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 90.3+ KB


In [226]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Extract features and target variable
features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
# Convert categorical variables to numerical
# df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

# Split data into training and prediction sets
train_df = df[df['Age'].notna()]
predict_df = df[df['Age'].isna()]

X_train = train_df[features]
y_train = train_df['Age']
X_predict = predict_df[features]

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# Predict the missing values
predicted_ages = model.predict(X_predict)

# Fill the missing values
df.loc[df['Age'].isna(), 'Age'] = predicted_ages

In [231]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    int64  
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 90.3+ KB


In [236]:
data_3 = df.drop(['Name',"Cabin","PassengerId","Ticket"], axis = 1)
data_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    int64  
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Embarked  889 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 62.5 KB


In [237]:
y = data_3.Survived
X  = data_3.drop(["Survived"], axis = 1)

In [238]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [239]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.788928 (0.039689)
LDA: 0.787559 (0.033908)
KNN: 0.689163 (0.039094)
CART: 0.770677 (0.046569)
NB: 0.780595 (0.027552)
SVM: 0.696146 (0.040781)


LR: c (0.036389)
LDA: 0.780575 (0.042415)
KNN: 0.669464 (0.039565)
CART: 0.739691 (0.060500)
NB: 0.777797 (0.029905)
SVM: 0.674980 (0.050749)
